In [1]:
import os
import sys 
import pandas as pd
from train_models import train_dl_model, train_transf_chik, LOOK_BACK, PREDICT_N

sys.path.append('../')
from lstm import apply_model

df = pd.read_csv('s_cities.csv', index_col = 'Unnamed: 0')

df = df.loc[~df.geocode.isin([3135050, 3143302, 1716109, 3304102, 3141405, 2905008, 29005008])]

#df = df.loc[df.geocode != 3168606]
#df = df.loc[]
#df = df.iloc[:1]
df.head()

,geocode,muni_name,year,peak_week,beta,gamma,R0,total_cases,alpha,sum_res,ep_ini,ep_end,ep_dur,uf,state,n_last_ep,year_first_ep,year_last_ep_before_2023,start_train_chik,end_train_chik
8,2303501,Cascavel,2023,21.031014,0.447854,0.300026,1.492714,196.791047,0.330079,1.124154,202302,202339,37,23,CE,3,2017,2022,2016-10-01,2022-11-01
86,2311405,Quixeramobim,2023,18.372554,0.533369,0.303351,1.758256,466.403595,0.431255,0.930381,202302,202332,30,23,CE,3,2016,2022,2015-10-01,2022-11-01
88,2306801,Jaguaribara,2023,16.407407,0.744166,0.309970,2.400772,302.644966,0.583467,1.126158,202309,202325,16,23,CE,1,2017,2017,2016-10-01,2017-11-01
122,2305407,Icó,2023,13.682026,0.429828,0.300005,1.432734,87.125102,0.302034,1.434513,202302,202334,32,23,CE,2,2017,2022,2016-10-01,2022-11-01
152,2313203,Tamboril,2023,7.898419,0.938429,0.300000,3.128098,614.430188,0.680317,0.896101,202303,202314,11,23,CE,2,2016,2022,2015-10-01,2022-11-01


,geocode,muni_name,year,peak_week,beta,gamma,R0,total_cases,alpha,sum_res,ep_ini,ep_end,ep_dur,uf,state,n_last_ep,year_first_ep,year_last_ep_before_2023,start_train_chik,end_train_chik
8,2303501,Cascavel,2023,21.031014,0.447854,0.300026,1.492714,196.791047,0.330079,1.124154,202302,202339,37,23,CE,3,2017,2022,2016-10-01,2022-11-01
86,2311405,Quixeramobim,2023,18.372554,0.533369,0.303351,1.758256,466.403595,0.431255,0.930381,202302,202332,30,23,CE,3,2016,2022,2015-10-01,2022-11-01
88,2306801,Jaguaribara,2023,16.407407,0.744166,0.309970,2.400772,302.644966,0.583467,1.126158,202309,202325,16,23,CE,1,2017,2017,2016-10-01,2017-11-01
122,2305407,Icó,2023,13.682026,0.429828,0.300005,1.432734,87.125102,0.302034,1.434513,202302,202334,32,23,CE,2,2017,2022,2016-10-01,2022-11-01
152,2313203,Tamboril,2023,7.898419,0.938429,0.300000,3.128098,614.430188,0.680317,0.896101,202303,202314,11,23,CE,2,2016,2022,2015-10-01,2022-11-01


### Train models

In [2]:
%%time
import pandas as pd
from train_models import train_dl_model, train_transf_chik
import os

end_date = '2023-11-01'
RETRAIN = True

for city, ini_date, end_date_train in zip(
    df.geocode, df.start_train_chik, df.end_train_chik):
    print(f"Training {city} model")

    # train the dengue model
    if RETRAIN or not os.path.exists(f'../saved_models/lstm/trained_{city}_dengue_msle.keras'):
        train_dl_model(city,   doenca = 'dengue', end_date_train = '2022-11-01' , end_date = end_date, plot = False)
    
        
    # apply transfer
    if  RETRAIN or not os.path.exists(f'../saved_models/lstm/trained_{city}_chik_tranf_msle.keras'):
        train_transf_chik(city, ini_date = ini_date, end_date_train = end_date_train, end_date = end_date, plot =False)
    
    
    
    # train the chik model
    if RETRAIN or not os.path.exists(f'../saved_models/lstm/trained_{city}_chik_msle.keras'):
        train_dl_model(city,   doenca = 'chik', end_date_train = '2022-11-01' , end_date = end_date, plot = False)


Training 2303501 model
Training 2303501 model
Compilation Time :  0.0019009113311767578
Model: "model"
Compilation Time :  0.0019009113311767578
Model: "model"
_________________________________________________________________
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 12, 70)]          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 12, 8)            5056      
 nal)                                                            
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 12, 70)]          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 12, 8)            5056      
 nal)                                                            
                                                

2024-01-15 10:45:12.565851: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-01-15 10:45:12.587020: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1780] (One-time warning): Not using XLA:CPU for cluster.

If you want XLA:CPU, do one of the following:

 - set the TF_XLA_FLAGS to include "--tf_xla_cpu_global_jit", or
 - set cpu_global_jit to true on this session's OptimizerOptions, or
 - use experimental_jit_scope, or
 - use tf.function(jit_compile=True).

To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a
proper command-line flag, not via TF_XLA_FLAGS).
2024-01-15 10:45:12.565851: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-01-15 10:45:12.587020: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1780] (One-time warning): Not using XLA:CPU for cluster.

If you want XLA:CPU, do one of the following:

 - set the TF_XLA_FLAGS to include "--tf_xla_cpu_glo

Training 2311405 model
Training 2311405 model
Compilation Time :  0.0016498565673828125
Model: "model_1"
_________________________________________________________________
Compilation Time :  0.0016498565673828125
Model: "model_1"
 Layer (type)                Output Shape              Param #   
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 12, 53)]          0         
                                                                 
 input_2 (InputLayer)        [(None, 12, 53)]          0         
 bidirectional_1 (Bidirectio  (None, 12, 8)            3968      
                                                                 
 nal)                                                            
 bidirectional_1 (Bidirectio  (None, 12, 8)            3968      
                                                                 
 nal)                                       

 dropout_1 (Dropout)         (None, 12, 8)             0         
                                                                 
 lstm_1 (LSTM)               (None, 12, 8)             544       
                                                                 
 dropout_2 (Dropout)         (None, 12, 8)             0         
                                                                 
                                                                 
 lstm_2 (LSTM)               (None, 8)                 544       
 dropout_3 (Dropout)         (None, 8)                 0         
                                                                 
                                                                 
 dropout_3 (Dropout)         (None, 8)                 0         
 dense (Dense)               (None, 4)                 36        
                                                                 
                                                                 
 dense (De

/Users/eduardoaraujo/Documents/Github/transfer-learning-forecast_vem_ai_a_versao_final/notebooks/../preprocessing.py:179: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  norm_df[f'diff_{i}'] = np.concatenate(([np.nan], np.diff(norm_df[f'{i}'], 1)), axis = 0)
/Users/eduardoaraujo/Documents/Github/transfer-learning-forecast_vem_ai_a_versao_final/notebooks/../preprocessing.py:179: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  norm_df[f'diff_{i}'] = np.concatenate(([np.nan], np.diff(norm_df[f'{i}'], 1)), axis = 0)
/Users/eduardoaraujo

...

/Users/eduardoaraujo/Documents/Github/transfer-learning-forecast_vem_ai_a_versao_final/notebooks/../preprocessing.py:179: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  norm_df[f'diff_{i}'] = np.concatenate(([np.nan], np.diff(norm_df[f'{i}'], 1)), axis = 0)
/Users/eduardoaraujo/Documents/Github/transfer-learning-forecast_vem_ai_a_versao_final/notebooks/../preprocessing.py:179: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  norm_df[f'diff_{i}'] = np.concatenate(([np.nan], np.diff(norm_df[f'{i}'], 1)), axis = 0)
/Users/eduardoaraujo

/Users/eduardoaraujo/Documents/Github/transfer-learning-forecast_vem_ai_a_versao_final/notebooks/../preprocessing.py:179: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  norm_df[f'diff_{i}'] = np.concatenate(([np.nan], np.diff(norm_df[f'{i}'], 1)), axis = 0)
/Users/eduardoaraujo/Documents/Github/transfer-learning-forecast_vem_ai_a_versao_final/notebooks/../preprocessing.py:179: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  norm_df[f'diff_{i}'] = np.concatenate(([np.nan], np.diff(norm_df[f'{i}'], 1)), axis = 0)
/Users/eduardoaraujo

Training 2931350 model
Training 2931350 model
Compilation Time :  0.0016438961029052734
Model: "model_27"
Compilation Time :  0.0016438961029052734
Model: "model_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_28 (InputLayer)       [(None, 12, 36)]          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 12, 8)            2880      
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_28 (InputLayer)       [(None, 12, 36)]          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 12, 8)            2880      
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 12, 8)

                                                                 
 lstm_1 (LSTM)               (None, 12, 8)             544       
                                                                 
 dropout_2 (Dropout)         (None, 12, 8)             0         
                                                                 
 lstm_2 (LSTM)               (None, 8)                 544       
                                                                 
 lstm_2 (LSTM)               (None, 8)                 544       
                                                                 
 dropout_3 (Dropout)         (None, 8)                 0         
                                                                 
 dense (Dense)               (None, 4)                 36        
                                                                 
 dropout_3 (Dropout)         (None, 8)                 0         
                                                                 
 dense (De

Compilation Time :  0.0017337799072265625
Model: "model_32"
_________________________________________________________________
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_33 (InputLayer)       [(None, 12, 19)]          0         
 Layer (type)                Output Shape              Param #   
                                                                 
 input_33 (InputLayer)       [(None, 12, 19)]          0         
 bidirectional_1 (Bidirectio  (None, 12, 8)            1792      
                                                                 
 nal)                                                            
 bidirectional_1 (Bidirectio  (None, 12, 8)            1792      
                                                                 
 nal)                                                            
 dropout_1 (Dropout)         (None, 12, 8)             0         
                

## Apply models

In [ ]:
%%time 

for city, ini_date, end_date_train in zip(
    df.geocode, df.start_train_chik, df.end_train_chik): 
    

    # apply dl dengue on chik data 
    metrics = apply_model(city, ini_date = ini_date, 
                    end_date = end_date, look_back = LOOK_BACK, end_train_date =  '2022-11-01', batch_size = 1, 
                    predict_n = PREDICT_N,  ratio = None,
                    label_pred= 'chik_dengue_pred',
                    model_name = f'trained_{city}_dengue_msle', 
                    filename = f'../data/chik_{city}_cluster.csv', plot = False)
    

    # apply transfer 

    metrics = apply_model(city, ini_date = ini_date, 
                    end_date = end_date, look_back = LOOK_BACK, end_train_date = '2022-11-01', batch_size = 1, 
                    predict_n = PREDICT_N,  ratio = None,
                    label_pred= 'chik_transf_pred',
                    model_name = f'trained_{city}_chik_transf_msle', 
                    filename = f'../data/chik_{city}_cluster.csv', plot = False)
    

    # apply dl chik on chik 

    metrics = apply_model(city, ini_date = ini_date, 
                    end_date = end_date, look_back = LOOK_BACK, end_train_date =  '2022-11-01', batch_size = 1, 
                    predict_n = PREDICT_N,  ratio = None,
                    label_pred= 'chik_pred',
                    model_name = f'trained_{city}_chik_msle', 
                    filename = f'../data/chik_{city}_cluster.csv', plot = False)
    
